# Mission - Analysez des images médicales avec des méthodes semi-supervisées

Vous êtes Data Scientist junior spécialisé en Computer Vision au sein de CurelyticsIA, une startup innovante dans le domaine de la e-santé. L’entreprise développe des solutions basées sur l’intelligence artificielle pour assister les professionnels de santé dans l’analyse d’images médicales, en particulier des IRM.
 
Dans le cadre d’un nouveau projet R&D, CurelyticsIA souhaite explorer la possibilité d’automatiser la détection de tumeurs du cerveau. Un ensemble conséquent de radios a été collecté : la majorité de ces images ne dispose d’aucun étiquetage, tandis qu’un sous-ensemble limité a été annoté par des radiologues experts.
 
Vous êtes chargé de concevoir une première exploration analytique du jeu de données. Plus précisément, votre mission est de :
- Explorer les images et extraire des caractéristiques visuelles via un modèle pré-entraîné ;
- Appliquer des méthodes de clustering pour identifier des structures ou regroupements dans les données ;
- Mettre en œuvre une méthode d’apprentissage semi-supervisé à partir des quelques étiquettes disponibles ;
- Synthétiser vos résultats, formuler des recommandations, et les présenter à votre équipe projet.

**Mail à prendre en compte :**

Comme discuté lors de notre dernière réunion, tu es assigné à la première phase du projet BrainScanAI. Tu trouveras en pièce jointe un fichier zip contenant :
- Le jeu de données de radiographies (en format PNG + métadonnées anonymisées),
- Une documentation technique sur le format des images ;
- Une liste restreinte de labels annotés par nos partenaires hospitaliers (normal/cancéreux). 

Pour info, notre budget actuel pour la labellisation par IA est de 300 euros pour ce dataset. 

Tes objectifs :
1) Extraire des caractéristiques visuelles pertinentes à l’aide d’un modèle pré-entraîné (type ResNet ou équivalent).
2) Réaliser un clustering exploratoire pour identifier des regroupements naturels.
3) Mettre en œuvre une méthode semi-supervisée en exploitant les labels partiels pour prédire les étiquettes manquantes.
4) Proposer des livrables au format Notebook contenant :
    - l’extraction des features
    - le preprocessing adapté au(x) modèle(s) utilisés
    - l’analyse non-supervisée (.ipynb)
    - l’entraînement de modèles de clustering
    - l’approche semi-supervisée (.ipynb)
 
Ces livrables doivent être accompagnés d’un support de présentation proposant des recommandations techniques pour un passage à l’échelle (budget de 5 000 euros pour 4 millions d’images à labelliser). Est-ce que ce passage te paraît faisable et si oui, sous quelles conditions ?

5) Rédiger une synthèse de ton approche et de tes résultats dans un support de présentation. Les contraintes :
    - Travailler en Python.
    - Tester plusieurs algorithmes.
    - Avoir des métriques pertinentes en fonction de l’erreur la plus importante (F1, Acc, Précision, ou autre ?).
    - Clairement définir ce que tu considères comme un objectif atteint (“definition of done”).


## Étape 4 - Appliquez une méthode semi supervisée

Entraîner un modèle de type CNN sur votre jeu de données “faiblement” labellisé dans un premier temps puis et évaluer ses performances. Poursuivez ensuite l’entraînement de ce même modèle sur le jeu de données “fortement” labellisé. Comparer ensuite la différence de performance entre entraînement supervisé (modèle entraîné sur le jeu de données “fortement” labellisé uniquement) et semi-supervisé (entraîné sur les 2 jeux de données).
 
**Prérequis**
- Avoir préparé un ensemble labellisé et un ensemble non labellisé

**Résultat attendu**
- Modèle entraîné et validé avec les métriques choisies (accuracy, F1-score ou autre ?)

**Recommandations**
- Réaliser un split train/test équilibré
- Pour chaque évaluation des performances: attention à ce que le jeu de test soir bien des données jamais vues par le modèle évalué lors de son entraînement.
- Utiliser des visualisations bien lisibles pour analyser les performances.

**Outils**
- Torchvision / TenforFlow / Transformers / Numpy / Pandas

## Importation des librairies

In [30]:
# Librairies de base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librairies spécifiques
import os # permet de travailler avec le système de fichiers
from PIL import Image # ouvrir et manipuler des images
import glob

# Librairies PyTorch
import torchvision
import torch
from torchvision import transforms # pour effectuer les changements de format, entre autres
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Librairies Scikit-learn
from sklearn.metrics import accuracy_score, f1_score, recall_score

# Choisit automatiquement GPU si disponible, sinon CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Enregistrement des dataset

In [41]:
import glob
import pandas as pd
from PIL import Image
import torch
from torchvision import transforms
from sklearn.model_selection import train_test_split

# Chargement des chemins vers les images labellisées

cancer_paths = glob.glob("../mri_dataset_brain_cancer_oc/avec_labels/cancer/*")
normal_paths = glob.glob("../mri_dataset_brain_cancer_oc/avec_labels/normal/*")

# On regroupe les chemins
paths_labeled = cancer_paths + normal_paths

# Création des labels : 1 = cancer, 0 = normal
y_labeled = [1] * len(cancer_paths) + [0] * len(normal_paths)


# Chargement des pseudo-labels

pseudo_df = pd.read_csv("../data/processed/pseudo_labels.csv")

# On récupère les chemins des images non labellisées dans le même ordre
unlabeled_paths = sorted(glob.glob("../mri_dataset_brain_cancer_oc/sans_label/*"))

# Vérification de cohérence
assert len(unlabeled_paths) == len(pseudo_df), \
       "Erreur : nb d'images ≠ nb de pseudo-labels"

pseudo_labels = pseudo_df["pseudo_label"].values.tolist()

# Transformations à appliquer aux images

# Resize → Tensor → normalisation 0-1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Chargement effectif des images labellisées

print("Chargement des images labellisées...")

X_labeled = torch.stack([
    transform(Image.open(path).convert("RGB"))
    for path in paths_labeled
])

y_labeled = torch.tensor(y_labeled)

print(f"Images labellisées chargées : {X_labeled.shape}")


# Chargement des images avec pseudo-labels

print("Chargement des images pseudo-labellisées...")

X_unlabeled = torch.stack([
    transform(Image.open(path).convert("RGB"))
    for path in unlabeled_paths
])

pseudo_labels = torch.tensor(pseudo_labels)

print(f"Images pseudo-labellisées chargées : {X_unlabeled.shape}")


# Train/test split sur le dataset supervisé uniquement

X_train_labeled, X_test_labeled, y_train_labeled, y_test_labeled = train_test_split(
    X_labeled,
    y_labeled,
    test_size=0.2,
    stratify=y_labeled,   # conserve le même ratio cancer/normal
    random_state=42
)

print("\n Découpage terminé.")
print(f"Train supervisé : {X_train_labeled.shape}")
print(f"Test  supervisé : {X_test_labeled.shape}")


# 7) Résumé global

print("\n RÉSUMÉ DES TENSEURS DISPONIBLES :")
print(f" X_train_labeled  : {X_train_labeled.shape}")
print(f" y_train_labeled  : {y_train_labeled.shape}")
print(f" X_test_labeled   : {X_test_labeled.shape}")
print(f" y_test_labeled   : {y_test_labeled.shape}")
print("----------------------------------------")
print(f" X_unlabeled      : {X_unlabeled.shape}")
print(f" pseudo_labels    : {pseudo_labels.shape}")


Chargement des images labellisées...
Images labellisées chargées : torch.Size([100, 3, 224, 224])
Chargement des images pseudo-labellisées...
Images pseudo-labellisées chargées : torch.Size([1406, 3, 224, 224])

 Découpage terminé.
Train supervisé : torch.Size([80, 3, 224, 224])
Test  supervisé : torch.Size([20, 3, 224, 224])

 RÉSUMÉ DES TENSEURS DISPONIBLES :
 X_train_labeled  : torch.Size([80, 3, 224, 224])
 y_train_labeled  : torch.Size([80])
 X_test_labeled   : torch.Size([20, 3, 224, 224])
 y_test_labeled   : torch.Size([20])
----------------------------------------
 X_unlabeled      : torch.Size([1406, 3, 224, 224])
 pseudo_labels    : torch.Size([1406])


## Création de notre modèle CNN simple

In [32]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Première couche de convolution : 3 canaux RGB -> 32 / détecte bords, intensités
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)

        # Pooling pour réduire la taille de l'image
        self.pool = nn.MaxPool2d(2,2)

        # Deuxième convolution : 32 -> 64 / détecte formes plus larges
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Troisième convolution : 64 -> 128 / détecte motifs médicaux
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Couche fully connected : calculée pour image 224*224 -> 28x28 après 3 poolings
        self.fc1 = nn.Linear(128 * 28 * 28, 256)

        # Dropout pour réduire l'overfitting (important sur un petit dataset)
        self.dropout = nn.Dropout(0.3)

        # Dernière couche : 256 -> 1 (classification binaire)
        self.fc2 = nn.Linear(256,1)

        # La fonction qui définit comment le modèle traite une image.
    def forward(self, x):

        # Bloc 1 : conv -> relu -> pool
        x = self.pool(F.relu(self.conv1(x)))

        # Bloc 2
        x = self.pool(F.relu(self.conv2(x)))

        # Bloc 3
        x = self.pool(F.relu(self.conv3(x)))

        # Aplatit le tenseur pour la couche fully connected
        x = x.view(x.size(0), -1)

        # Dense layer + dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Sortie sigmoïde pour une probabilité entre 0 et 1
        x = torch.sigmoid(self.fc2(x))
        return x

#### Création des dataloaders

In [33]:
# Batch size pour entraîner par petits lots
batch_size = 32

# Dataset supervisé (vrais labels) ENTIEREMENT basé sur les données d'entraînement
dataset_sup = TensorDataset(X_train_labeled, y_train_labeled)

# Dataset pseudo-labellisé — ne change pas car X_unlabeled ne fait pas partie du split
dataset_pseudo = TensorDataset(X_unlabeled, pseudo_labels)

# Dataset fine-tuning (mêmes vraies données que supervisé → X_train)
dataset_ft = TensorDataset(X_train_labeled, y_train_labeled)

# Dataset test (utilise le split test)
dataset_test = TensorDataset(X_test_labeled, y_test_labeled)

# DataLoaders (chargent les données par batch)
loader_sup = DataLoader(TensorDataset(X_train_labeled, y_train_labeled),
                        batch_size=batch_size, shuffle=True)

loader_test = DataLoader(TensorDataset(X_test_labeled, y_test_labeled),
                         batch_size=batch_size, shuffle=False)

loader_pseudo = DataLoader(TensorDataset(X_unlabeled, pseudo_labels),
                           batch_size=batch_size, shuffle=True)

loader_ft = DataLoader(TensorDataset(X_train_labeled, y_train_labeled),
                       batch_size=batch_size, shuffle=True)



#### Entraînement supervisé

In [34]:
def train_supervised(model, loader, epochs=10, lr=1e-4):
    
    # Envoie le modèle sur GPU/CPU
    model = model.to(device)

    # Perte pour binaire
    criterion = nn.BCELoss()

    # Optimiseur Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    losses = []

    for epoch in range(epochs):

        model.train()  # Mode entraînement
        total_loss = 0

        # On parcourt les batches
        for X, y in loader:

            X = X.to(device)                       # Images sur GPU/CPU
            y = y.float().to(device).view(-1, 1)   # Labels → float + reshape

            optimizer.zero_grad()                  # Reset gradients
            pred = model(X)                        # Prédiction du CNN
            loss = criterion(pred, y)              # Calcul perte
            loss.backward()                        # Backpropagation
            optimizer.step()                       # Mise à jour des poids

            total_loss += loss.item()

        # Sauvegarde moyenne des pertes
        losses.append(total_loss / len(loader))
        print(f"[SUP] Epoch {epoch+1}/{epochs} - Loss: {losses[-1]:.4f}")

    return model, losses


#### Pré-entraînement sur pseudo-labels

In [35]:
def pretrain_pseudo(model, loader, epochs=10, lr=1e-4):

    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        for X, y in loader:

            X = X.to(device)
            y = y.float().to(device).view(-1, 1)
            
            optimizer.zero_grad()
            loss = criterion(model(X), y)
            loss.backward()
            optimizer.step()

        print(f"[PSEUDO] Epoch {epoch+1}/{epochs}")

    return model


#### Fine-tuning sur vrais labels

In [36]:
def finetune(model, loader, epochs=5, lr=5e-5):

    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        for X, y in loader:
            X = X.to(device)
            y = y.float().to(device).view(-1,1)
            
            optimizer.zero_grad()
            loss = criterion(model(X), y)
            loss.backward()
            optimizer.step()

        print(f"[FINETUNE] Epoch {epoch+1}/{epochs}")

    return model

#### Évaluation

In [37]:
def evaluate(model, loader):
    model.eval()

    preds = []
    truths = []

    with torch.no_grad():  # Pas de gradient en test
        for X, y in loader:

            X = X.to(device)
            pred = model(X).cpu().numpy()     # Sortie du modèle
            pred = (pred > 0.5).astype(int)   # Prob → classe (0/1)

            preds.extend(pred.flatten().tolist())
            truths.extend(y.numpy().astype(int).tolist())

    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds)
    rec = recall_score(truths, preds)

    return acc, f1, rec


#### Supervisé

In [38]:
model_sup = SimpleCNN()
model_sup, losses_sup = train_supervised(model_sup, loader_sup)
acc_sup, f1_sup, rec_sup = evaluate(model_sup, loader_test)

[SUP] Epoch 1/10 - Loss: 0.7099
[SUP] Epoch 2/10 - Loss: 0.6686
[SUP] Epoch 3/10 - Loss: 0.6340
[SUP] Epoch 4/10 - Loss: 0.5858
[SUP] Epoch 5/10 - Loss: 0.5492
[SUP] Epoch 6/10 - Loss: 0.5098
[SUP] Epoch 7/10 - Loss: 0.4674
[SUP] Epoch 8/10 - Loss: 0.4489
[SUP] Epoch 9/10 - Loss: 0.4189
[SUP] Epoch 10/10 - Loss: 0.4221


#### Semi-supervisé

In [39]:
model_semi = SimpleCNN()
model_semi = pretrain_pseudo(model_semi, loader_pseudo)
model_semi = finetune(model_semi, loader_ft)
acc_semi, f1_semi, rec_semi = evaluate(model_semi, loader_test)

[PSEUDO] Epoch 1/10
[PSEUDO] Epoch 2/10
[PSEUDO] Epoch 3/10
[PSEUDO] Epoch 4/10
[PSEUDO] Epoch 5/10
[PSEUDO] Epoch 6/10
[PSEUDO] Epoch 7/10
[PSEUDO] Epoch 8/10
[PSEUDO] Epoch 9/10
[PSEUDO] Epoch 10/10
[FINETUNE] Epoch 1/5
[FINETUNE] Epoch 2/5
[FINETUNE] Epoch 3/5
[FINETUNE] Epoch 4/5
[FINETUNE] Epoch 5/5


#### Tableau final comparatif

In [40]:
df_results = pd.DataFrame({
    "Méthode": ["Supervisé", "Semi-supervisé"],
    "Accuracy": [acc_sup, acc_semi],
    "F1": [f1_sup, f1_semi],
    "Recall": [rec_sup, rec_semi]
})

df_results

,Méthode,Accuracy,F1,Recall
0,Supervisé,0.85,0.842105,0.8
1,Semi-supervisé,0.80,0.777778,0.7
